## RPFS C&CG (Cmax objective) - Data treatment of result files 

In [ ]:
import pandas as pd
import numpy as np
import os, fnmatch
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
import glob
import os
from pathlib import Path

%matplotlib inline

In [ ]:
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

### List files in the output folder 

In [ ]:
rootfolder = os.path.join('..', 'pfsp_experiments', 'run_ccg_rpfs_cmax_global')
file_list = glob.glob(os.path.join(rootfolder, '*.csv'), recursive=True)
file_list

### Read all the CSV files 

In [ ]:
# Alternative script to treat files with incorrect number of coluns or faulty lines
def alternative_csv_reader(filename, delimiter=',', header=0, names=None):
    with open(filename, 'r') as file:
        lines = file.readlines() 
        count = 1
        line_list = []
        num_columns = 23
        for line_num, line in enumerate(lines):  # Strips the newline character 
            #print("line{}: {}".format(count, line.strip())) 
            nc = len(line.split(','))
            if 'executionId,' in line:
                #num_columns = nc
                if nc == num_columns - 1 and line_num == 0:
                    line_list.append(line.replace('\n', '') + ',cut_count\n')
                # end if
                print('Detected {0} columns in CSV file.'.format(nc))
            else:
                if 'none,' in line:
                    if nc == num_columns - 1:
                        line_list.append(line.replace('\n', '') + ',\n')
                    elif nc == num_columns:
                        line_list.append(line)
                    elif nc > num_columns:  # treat strange truncated lines
                        line = line[line.rfind('none,'):]
                        nc = len(line.split(','))
                        #if nc >= num_columns:
                        #    print('WARN: truncating line {0}, for having more columns than expected.'.format(count))
                        #    line_list.append(line)
                        #else:
                        print('WARN: Ignoring line {0}, since it has {1} columns, instead of {2}: '.format(count, nc, num_columns), line)
                    else:  # Ignore line
                        print('WARN: Ignoring line {0}: '.format(count), line)  
                else:  # Ignore line
                    print('WARN: Ignoring line {0}: '.format(count), line)
            count += 1
        # assert all lines have the same number of columns
        count = 1
        for line in line_list:
            nc = len(line.split(','))
            if nc < num_columns:
                print('ERROR: Line {0} has {1} columns, instead of {2}: '.format(count, nc, num_columns), line)
            count += 1
        text_data = StringIO(''.join(line_list))
        #print('line_list: ', str(line_list))
        #print('text_data: ', text_data)
        df = pd.read_csv(text_data, delimiter=delimiter, header=header, names=names)
        return df

### Process all CSV files and append all data to a single dataframe (one per solution method: Wilson, Wagner) 

In [ ]:
%%time

dfdict = dict()
for filepath in file_list:
    print('Processing file ', filepath)
    try:
        df_ = pd.read_csv(filepath, delimiter=',', header=0, names=['executionId','ub_name','instance_name','alpha','n','m','budget_Gamma','Gamma_abs','cmax','permutation','time_spent','time_to_best_sol','mp_total_time','sp_total_time','iterations','num_visited_solutions','num_improvements','is_optimal','validated','gap','lb','cost','cmax_validation','cut_count'])
    except:  # try alternative method to read csv lines
        df_ = alternative_csv_reader(filepath, delimiter=',', header=0, names=['executionId','ub_name','instance_name','alpha','n','m','budget_Gamma','Gamma_abs','cmax','permutation','time_spent','time_to_best_sol','mp_total_time','sp_total_time','iterations','num_visited_solutions','num_improvements','is_optimal','validated','gap','lb','cost','cmax_validation','cut_count'])
    filename = filepath[filepath.rfind(os.path.sep)+1:]
    if 'gamma' in filename or 'partialresults' in filename:  # skip result files with partial results
        continue
    if filename.find('_instance') > 0:
        modelname = filename[len('separation_cmax_'):filename.find('_instance')]
    else:
        if filename.find('_ying') > 0:
            modelname = filename[len('separation_cmax_'):filename.find('_ying')]
        elif filename.find('_tail') > 0:
            modelname = filename[len('separation_cmax_'):filename.find('_tail')]
        else:
            continue
    print('Read results for model ' + modelname)
    if modelname in dfdict:
        dfdict[modelname] = pd.concat([dfdict[modelname], df_])
    else:
        dfdict[modelname] = df_.copy()

In [ ]:
dfdict

In [ ]:
dfdict['wilson'].info()

### Remove duplicated header rows from both dataframes 

In [ ]:
def find_invalid_values(df):
    all_invalid_values = set()
    for col in df:
        if col not in ['executionId','ub_name','instance_name','budget_Gamma','permutation','is_optimal','validated']:
            # 'alpha','n','m','cmax','time_spent','time_to_best_sol','iterations','num_visited_solutions','num_improvements','gap','lb','cost','cmax_dp'
            a = pd.to_numeric(df[col], errors='coerce')
            idx = a.isna()
            invalid_values = df.loc[idx][col].unique()
            all_invalid_values.update(invalid_values)
        #elif col in ['is_optimal','validated']
    print('Invalid values:', all_invalid_values)
    return all_invalid_values

In [ ]:
for key, df in dfdict.items():
    dfdict[key] = df[(df['executionId'] != 'executionId')]
    print(key, dfdict[key].dtypes)

### Convert column types from object 

In [ ]:
def convert_column_types(df):
    for col in df:
        if col in ['alpha','n','m','cmax','budget_Gamma','Gamma_abs','time_spent','time_to_best_sol','iterations','num_visited_solutions','num_improvements','gap','lb','cost','cmax_validation', 'mp_total_time', 'sp_total_time', 'seq']:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        elif col in ['is_optimal','validated']:
            df[col] = df[col].astype('bool')
    return df

In [ ]:
%%time
for key, df in dfdict.items():
    dfdict[key] = convert_column_types(df)
    print(key, dfdict[key].dtypes)

### Trim existing string columns 

In [ ]:
def trim_all_columns(df):
    """
    Trim whitespace from ends of each value across all series in dataframe
    """
    trim_strings = lambda x: x.strip() if isinstance(x, str) else x
    return df.applymap(trim_strings)

In [ ]:
for key, df in dfdict.items():
    dfdict[key] = trim_all_columns(df)
    print(key)

### Include a column with the name of the underlying C&CG MILP Model

In [ ]:
for key, df in dfdict.items():
    dfdict[key]['model'] = key

### Include a column with the name of the instance type (ying or tail)

In [ ]:
for key, df in dfdict.items():
    print(key, df)
    dfdict[key]['instance_type'] = df['instance_name'].apply(lambda x: 'tail' if ('tail' in x) else 'ying')

### Include a column with the budget parameter value in %

In [ ]:
for key, df in dfdict.items():
    dfdict[key]['Gamma%'] = df['budget_Gamma']

### Concatenate dataframes 

In [ ]:
df = pd.concat(list(dfdict.values()))
df.head(4)

### Check for duplicated values

In [ ]:
df[df.duplicated()]

### Fix instance names 

The original instance names, in the instance file zip, were assembled incorrectly.

The problem lies in the alpha percentage. We are now going to fix this issue.

In [ ]:
df['n_str'] = df['n'].astype(str).str.zfill(3)
df['instance_name_short'] = df['instance_name'].apply(lambda x: x[:x.find('_cmax_inputs')] if '_cmax_inputs' in x else x[:x.find('.txt')])
df.loc[(df['instance_type'] == 'tail'), 'seq'] = df.loc[(df['instance_type'] == 'tail'), 'instance_name'].apply(lambda x: x[x.find('tail')+len('tail'):x.find('_')] if '_cmax_inputs' in x else x[x.find('tail')+len('tail'):x.find('.')])
df.loc[(df['instance_type'] == 'ying'), 'seq'] = df.loc[(df['instance_type'] == 'ying'), 'instance_name'].apply(lambda x: x[x.find('_')-2:x.find('_')] if '_cmax_inputs' in x else x[x.find('.')-2:x.find('.')])
#df['alpha_str'].loc[(df['m'] == 2)] = df.loc[(df['m'] == 2), 'instance_name'].apply(lambda x: x[x.rfind('_')+1:] if '_cmax_inputs' in x else x)
df['alpha_str'] = df['instance_name_short'].apply(lambda x: x[x.rfind('_')+1:] if '_' in x else 'na')
df.loc[(df['alpha_str'] == 'na'), 'alpha_str'] = df.loc[(df['alpha_str'] == 'na'), 'alpha'].astype(str)


In [ ]:
df['alpha_str'].unique()

In [ ]:
df['n_str'].unique()

### Round columns containing time (in seconds) 

In [ ]:
df['time_spent'] = df['time_spent'].round(2)
df['time_to_best_sol'] = df['time_to_best_sol'].round(2)
df['mp_total_time'] = df['mp_total_time'].round(2)
df['sp_total_time'] = df['sp_total_time'].round(2)

### Drop duplicate rows, preserving the newest (last) ones

In [ ]:
key_columns = ['model', 'n', 'm', 'alpha_str', 'seq', 'budget_Gamma', 'Gamma_abs', 'instance_type', 'instance_name']
df[df.duplicated(subset=key_columns)].sort_values(by=key_columns)

In [ ]:
df.drop_duplicates(subset=key_columns, keep='last', inplace=True)

### Sort data according to model, instance_name, alpha, n, m and Gamma and set index

In [ ]:
print('Sorting dataset...')
df = df.sort_values(key_columns)
display(df.dtypes)
df = df.set_index(key_columns, verify_integrity=True)
display(df.head(6))

### Find missing results, for a given value of alpha, n and m

For a given group of alpha, n, m and budget_Gamma, there should be 10 results.

First we will build a dataframe with the instances list and all required budget values.

In [ ]:
gamma_range = np.array(range(0, 101))[0:101:5]
gamma_range

In [ ]:
file_list1 = list(Path(os.path.join(rootfolder, 'instances', 'robust', 'ying', 'rob-pfsp-cmax')).rglob('*.txt'))
file_list2 = list(Path(os.path.join(rootfolder, 'instances', 'robust', 'taillard', 'rob-pfsp-cmax')).rglob('*.txt'))
file_list = file_list1 + file_list2
file_list

In [ ]:
data = []
rootfolder = os.getcwd()
file_set = set()
for path in file_list:
    instance_path = path.name
    #print('instance_path: ' + instance_path)    
    if '.txt' not in instance_path:
        continue
    if 'tail' in instance_path:
        if instance_path[:instance_path.find('_')] not in ['tail001', 'tail002', 'tail003', 'tail004', 'tail005', 'tail006', 'tail007', 'tail008', 'tail009', 'tail010']:
            #print(instance_path[:instance_path.find('_')])
            continue
    instance_name = instance_path[instance_path.rfind(os.path.sep)+1:]
    file_set.add(instance_name)
#print(file_set, file_set)
for instance_name in file_set:
    #print('instance_name: ' + instance_name)
    seq = instance_name[instance_name.find('_')-2:instance_name.find('_')]
    info = instance_name[instance_name.find('_')+1:]
    n = info[:info.find('_')]
    info = info[info.find('_')+1:]
    m = info[:info.find('_')]
    info = info[info.find('_')+1:]
    alpha = info[:info.find('_')]
    
    if '_cmax_inputs' in instance_name:
        instance_name_short = instance_name[:instance_name.find('_cmax_inputs')]
    else:
        instance_name_short = instance_name[:instance_name.find('.txt')]
    # end if
    if '_' in instance_name:
        alpha = instance_name_short[instance_name_short.rfind('_')+1:]
    else:
        alpha = instance_name_short[5:7]  # RB1501001.txt
    # end if
    
    instance_type = 'ying'
    if 'tail' in instance_name:
        instance_type = 'tail'
    else:
        if int(n) > 20:
            continue
    for gamma in gamma_range:
        budget_gamma = (gamma * (int(m) * int(n)) / 100.0)
        for model in list(dfdict.keys()):
            data.append([model, seq, alpha, int(n), int(m), budget_gamma, gamma, instance_type, instance_name])
df_instances = pd.DataFrame(data, columns=['model', 'seq', 'alpha_str', 'n', 'm', 'budget_Gamma', 'Gamma_abs', 'instance_type', 'instance_name'])
for col in df_instances:
    if col in ['n','m','budget_Gamma','Gamma_abs']:
        df_instances[col] = pd.to_numeric(df_instances[col], errors='coerce')
display(df_instances.dtypes)
df_instances = df_instances.set_index(['model', 'n', 'm', 'alpha_str', 'seq', 'budget_Gamma', 'Gamma_abs', 'instance_type', 'instance_name'], verify_integrity=True)
display(df_instances)


In [ ]:
df_ = df_instances.reset_index()
df_[df_['alpha_str'].isna()]

In [ ]:
df_instances.info()

In [ ]:
df.info()

Now, lets join the instances dataframe with the results one (left join).

In [ ]:
df_joined = df_instances.join(df, how='left')
df_joined

Now we will export to CSV a list with all rows with NaN values (missing experimental results).

In [ ]:
missing_df = df_joined[df_joined[['cmax']].isnull().any(axis=1)].reset_index()[['model', 'n', 'm', 'alpha_str', 'seq', 'budget_Gamma', 'Gamma_abs', 'instance_type', 'instance_name']]
outputfolder = os.path.join(os.getcwd(), 'results', 'consolidated')
if not os.path.exists(outputfolder):
    os.makedirs(outputfolder)
print('Saving file on folder: ' + outputfolder)
fname = os.path.join(outputfolder, 'RPFS_cmax_missing_results.xlsx')
missing_df.to_excel(fname)
print('Saved: ' + fname)

In [ ]:
df_grouped = df.groupby(['alpha', 'n', 'm', 'Gamma%']).agg({'executionId' : ['count']}).reset_index()
df_grouped.columns = [ ' '.join(str(i) for i in col) for col in df_grouped.columns]
#df_grouped.reset_index(inplace=True)
df_grouped

In [ ]:
perc_list = gamma_range
table = pd.pivot_table(df[(df['Gamma%'].isin(perc_list))], values='executionId', index=['instance_type', 'alpha_str', 'n', 'm', 'model'], columns=['Gamma%'], aggfunc='count', fill_value=0)
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(table)

### Export the dataset to CSV file 

In [ ]:
%%time

outputfolder = os.path.join(os.getcwd(), 'results', 'consolidated')
print('Saving file on folder: ' + outputfolder)
fname = os.path.join(outputfolder, 'RPFS_Cmax_CCG_all_results.csv')
df.to_csv(fname, sep=';')
print('Saved: ' + fname)
fname = os.path.join(outputfolder, 'RPFS_Cmax_CCG_all_results.pkl.gz')
df.to_pickle(fname)
print('Saved: ' + fname)

In [ ]:
df.reset_index().info()